In [1]:
import pickle
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
pd.options.display.max_rows = 999

# Generic Data Augmentation

# Baseline runs

In [2]:
base_dir = "/bigstor/zsarwar/Enola_Augmented/MT_Imagenet_dogs_101_637d23ae78e9c5b5ef63d577ea9250fc"

runs   = [
            "MT_Baseline_2bf6a5b5ad6241cc4d3d04e73af4f874/Metrics"
        ]   

stats_filename = "agg_class_stats.pkl"

dhard_base = "/bigstor/zsarwar/Enola_Augmented/MT_Imagenet_dogs_101_637d23ae78e9c5b5ef63d577ea9250fc/Datasets"

dhard_runs =  [
                "df_imagenet_dogs_val_test_empirical_2bf6a5b5ad6241cc4d3d04e73af4f874.pkl"
              ]

dhard_sub_runs = [run.replace("val_test_empirical", "val_test_sub_empirical") for run in dhard_runs]

# Load labels
val_path = "/bigstor/zsarwar/Enola_Augmented/MT_Imagenet_dogs_101_637d23ae78e9c5b5ef63d577ea9250fc/Datasets/df_imagenet_dogs_val_test.pkl"
df_val = pd.read_pickle(val_path)


# Start analysis for entire val set
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_acc = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_acc.append(stats['accuracy']['best'])

df_val = df_val.drop_duplicates('label')
df_val_classes = df_val[['class', 'label']]

label_list = list(df_val_classes['label'])

all_acc_list = []
for i in range(len(all_stats_acc)):
    accuracy_list = [all_stats_acc[i][cl] for cl in label_list]
    all_acc_list.append(accuracy_list)

for i in range(len(all_acc_list)):
    col_name = f"accuracy_{i}"
    df_val_classes.insert(2, col_name, all_acc_list[i], True)

df_base = df_val_classes.copy()

cols = [f"accuracy_{i}" for i in range(len(all_acc_list))]
mean_accs = []
for c in cols:
    t_acc = df_base[c].mean()
    mean_accs.append(t_acc)
print("Accruracy for each run : ", mean_accs)
mean_accs = np.asarray(mean_accs)
print("Mean accuracy across runs : ", np.mean(mean_accs))   

thrashed_classes = ['Welsh springer spaniel','Bedlington terrier','Pomeranian','pug, pug-dog','African hunting dog, hyena dog, Cape hunting dog, Lycaon pictus','Bernese mountain dog','Boston bull, Boston terrier','Leonberg','Samoyed, Samoyede','borzoi, Russian wolfhound']
df_thrashed = df_base[df_base['class'].isin(thrashed_classes)]

for c in cols:
    print(f"Mean Acc for thrashed classes : {df_thrashed[c].mean()}")

Accruracy for each run :  [78.73333333333333]
Mean accuracy across runs :  78.73333333333333
Mean Acc for thrashed classes : 60.0


# Random

# Augmented runs

In [7]:
base_dir = "/bigstor/zsarwar/Enola_Augmented/MT_Imagenet_dogs_101_637d23ae78e9c5b5ef63d577ea9250fc"

runs = [
    "MT_Augmented_47349fd8db84a902554ec40cfa9023db/Metrics"
]

stats_filename = "agg_class_stats.pkl"

# Load labels
val_path = "/bigstor/zsarwar/Enola_Augmented/MT_Imagenet_dogs_101_637d23ae78e9c5b5ef63d577ea9250fc/Datasets/df_imagenet_dogs_val_test.pkl"
df_val = pd.read_pickle(val_path)


# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_acc = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_acc.append(stats['accuracy'])

df_val = df_val.drop_duplicates('label')
df_val_classes = df_val[['class', 'label']]

label_list = list(df_val_classes['label'])

all_acc_list = []
for i in range(len(all_stats_acc)):
    accuracy_list = [all_stats_acc[i]['best'][cl] for cl in label_list]
    all_acc_list.append(accuracy_list)

for i in range(len(all_acc_list)):
    col_name = f"accuracy_best_{i}"
    df_val_classes.insert(2, col_name, all_acc_list[i], True)   

df_aug = df_val_classes.copy()
cols = [f"accuracy_best_{i}" for i in range(len(all_acc_list))]
mean_accs = []
for c in cols:
    t_acc = df_aug[c].mean()
    mean_accs.append(t_acc)
print("Accruracy for each run : ", mean_accs)
mean_accs = np.asarray(mean_accs)
print("Mean accuracy across runs : ", np.mean(mean_accs))



thrashed_classes = ['Welsh springer spaniel','Bedlington terrier','Pomeranian','pug, pug-dog','African hunting dog, hyena dog, Cape hunting dog, Lycaon pictus','Bernese mountain dog','Boston bull, Boston terrier','Leonberg','Samoyed, Samoyede','borzoi, Russian wolfhound']
df_thrashed = df_aug[df_aug['class'].isin(thrashed_classes)]

for c in cols:
    print(f"Mean Acc for thrashed classes : {df_thrashed[c].mean()}")

Accruracy for each run :  [79.95]
Mean accuracy across runs :  79.95
Mean Acc for thrashed classes : 67.8


# Analysis of DHard and DHard_sub

In [8]:
# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_accs_dhard = []
all_stats_accs_dhard_sub = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_accs_dhard.append(stats['accuracy']['dhard_best'])
    all_stats_accs_dhard_sub.append(stats['accuracy']['dhard_sub_best'])

dhard_accs = list(all_stats_accs_dhard[0].values()) 
dhard_sub_accs = list(all_stats_accs_dhard_sub[0].values()) 
new_cols = [dhard_accs, dhard_sub_accs]
df_thrashed.insert(2, 'dhard_accs', dhard_accs)
df_thrashed.insert(3, 'dhard_sub_accs', dhard_sub_accs)


In [9]:
df_thrashed

,class,label,dhard_accs,dhard_sub_accs,accuracy_best_0
index,,,,,
ILSVRC2012_val_00000057.JPEG,Pomeranian,107,40.000000,25.0,76.0
ILSVRC2012_val_00000078.JPEG,"African hunting dog, hyena dog, Cape hunting d...",119,40.000000,25.0,94.0
ILSVRC2012_val_00000220.JPEG,"pug, pug-dog",102,17.647058,50.0,68.0
ILSVRC2012_val_00000350.JPEG,"Boston bull, Boston terrier",44,27.272728,50.0,62.0
ILSVRC2012_val_00000590.JPEG,"Samoyed, Samoyede",106,27.272728,25.0,84.0
ILSVRC2012_val_00000883.JPEG,Bedlington terrier,30,14.285714,25.0,62.0
ILSVRC2012_val_00001203.JPEG,Leonberg,103,50.000000,50.0,92.0
ILSVRC2012_val_00001626.JPEG,"borzoi, Russian wolfhound",18,32.142857,50.0,60.0
ILSVRC2012_val_00002298.JPEG,Welsh springer spaniel,67,25.000000,25.0,50.0


In [10]:
print("Mean dhard performance", df_thrashed['dhard_accs'].mean())
print("Mean dhard_sub performance", df_thrashed['dhard_sub_accs'].mean())
print("Mean acc_improvement", df_thrashed['accuracy_best_0'].mean())

Mean dhard performance 27.63988628387451
Mean dhard_sub performance 32.5
Mean acc_improvement 67.8


# GradMatch

# Augmented runs

In [16]:
base_dir = "/bigstor/zsarwar/Enola_Augmented/MT_Imagenet_dogs_101_637d23ae78e9c5b5ef63d577ea9250fc"

runs = [
    "MT_Augmented_4de678ead23b82825430eab813a11971/Metrics"
]

stats_filename = "agg_class_stats.pkl"

# Load labels
val_path = "/bigstor/zsarwar/Enola_Augmented/MT_Imagenet_dogs_101_637d23ae78e9c5b5ef63d577ea9250fc/Datasets/df_imagenet_dogs_val_test.pkl"
df_val = pd.read_pickle(val_path)


# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_acc = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_acc.append(stats['accuracy'])

df_val = df_val.drop_duplicates('label')
df_val_classes = df_val[['class', 'label']]

label_list = list(df_val_classes['label'])

all_acc_list = []
for i in range(len(all_stats_acc)):
    accuracy_list = [all_stats_acc[i]['best'][cl] for cl in label_list]
    all_acc_list.append(accuracy_list)

for i in range(len(all_acc_list)):
    col_name = f"accuracy_best_{i}"
    df_val_classes.insert(2, col_name, all_acc_list[i], True)   

df_aug = df_val_classes.copy()
cols = [f"accuracy_best_{i}" for i in range(len(all_acc_list))]
mean_accs = []
for c in cols:
    t_acc = df_aug[c].mean()
    mean_accs.append(t_acc)
print("Accruracy for each run : ", mean_accs)
mean_accs = np.asarray(mean_accs)
print("Mean accuracy across runs : ", np.mean(mean_accs))



thrashed_classes = ['Welsh springer spaniel','Bedlington terrier','Pomeranian','pug, pug-dog','African hunting dog, hyena dog, Cape hunting dog, Lycaon pictus','Bernese mountain dog','Boston bull, Boston terrier','Leonberg','Samoyed, Samoyede','borzoi, Russian wolfhound']
df_thrashed = df_aug[df_aug['class'].isin(thrashed_classes)]

for c in cols:
    print(f"Mean Acc for thrashed classes : {df_thrashed[c].mean()}")

Accruracy for each run :  [80.68333333333334]
Mean accuracy across runs :  80.68333333333334
Mean Acc for thrashed classes : 69.2


# Analysis of DHard and DHard_sub

In [17]:
# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_accs_dhard = []
all_stats_accs_dhard_sub = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_accs_dhard.append(stats['accuracy']['dhard_best'])
    all_stats_accs_dhard_sub.append(stats['accuracy']['dhard_sub_best'])

dhard_accs = list(all_stats_accs_dhard[0].values()) 
dhard_sub_accs = list(all_stats_accs_dhard_sub[0].values()) 
new_cols = [dhard_accs, dhard_sub_accs]
df_thrashed.insert(2, 'dhard_accs', dhard_accs)
df_thrashed.insert(3, 'dhard_sub_accs', dhard_sub_accs)


In [18]:
df_thrashed

,class,label,dhard_accs,dhard_sub_accs,accuracy_best_0
index,,,,,
ILSVRC2012_val_00000057.JPEG,Pomeranian,107,40.000000,25.0,76.0
ILSVRC2012_val_00000078.JPEG,"African hunting dog, hyena dog, Cape hunting d...",119,60.000000,50.0,96.0
ILSVRC2012_val_00000220.JPEG,"pug, pug-dog",102,29.411764,25.0,74.0
ILSVRC2012_val_00000350.JPEG,"Boston bull, Boston terrier",44,31.818182,50.0,66.0
ILSVRC2012_val_00000590.JPEG,"Samoyed, Samoyede",106,63.636364,75.0,90.0
ILSVRC2012_val_00000883.JPEG,Bedlington terrier,30,14.285714,25.0,64.0
ILSVRC2012_val_00001203.JPEG,Leonberg,103,37.500000,50.0,88.0
ILSVRC2012_val_00001626.JPEG,"borzoi, Russian wolfhound",18,32.142857,50.0,60.0
ILSVRC2012_val_00002298.JPEG,Welsh springer spaniel,67,21.875000,25.0,48.0


In [19]:
print("Mean dhard performance", df_thrashed['dhard_accs'].mean())
print("Mean dhard_sub performance", df_thrashed['dhard_sub_accs'].mean())
print("Mean acc_improvement", df_thrashed['accuracy_best_0'].mean())

Mean dhard performance 34.73365468978882
Mean dhard_sub performance 40.0
Mean acc_improvement 69.2


# Unicom

# Augmented runs

In [12]:
base_dir = "/bigstor/zsarwar/Enola_Augmented/MT_Imagenet_dogs_101_637d23ae78e9c5b5ef63d577ea9250fc"

runs = [
    "MT_Augmented_254bdaf0cc894706e21dd898f6527932/Metrics"
]

stats_filename = "agg_class_stats.pkl"

# Load labels
val_path = "/bigstor/zsarwar/Enola_Augmented/MT_Imagenet_dogs_101_637d23ae78e9c5b5ef63d577ea9250fc/Datasets/df_imagenet_dogs_val_test.pkl"
df_val = pd.read_pickle(val_path)


# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_acc = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_acc.append(stats['accuracy'])

df_val = df_val.drop_duplicates('label')
df_val_classes = df_val[['class', 'label']]

label_list = list(df_val_classes['label'])

all_acc_list = []
for i in range(len(all_stats_acc)):
    accuracy_list = [all_stats_acc[i]['best'][cl] for cl in label_list]
    all_acc_list.append(accuracy_list)

for i in range(len(all_acc_list)):
    col_name = f"accuracy_best_{i}"
    df_val_classes.insert(2, col_name, all_acc_list[i], True)   

df_aug = df_val_classes.copy()
cols = [f"accuracy_best_{i}" for i in range(len(all_acc_list))]
mean_accs = []
for c in cols:
    t_acc = df_aug[c].mean()
    mean_accs.append(t_acc)
print("Accruracy for each run : ", mean_accs)
mean_accs = np.asarray(mean_accs)
print("Mean accuracy across runs : ", np.mean(mean_accs))



thrashed_classes = ['Welsh springer spaniel','Bedlington terrier','Pomeranian','pug, pug-dog','African hunting dog, hyena dog, Cape hunting dog, Lycaon pictus','Bernese mountain dog','Boston bull, Boston terrier','Leonberg','Samoyed, Samoyede','borzoi, Russian wolfhound']
df_thrashed = df_aug[df_aug['class'].isin(thrashed_classes)]

for c in cols:
    print(f"Mean Acc for thrashed classes : {df_thrashed[c].mean()}")

Accruracy for each run :  [80.63333333333334]
Mean accuracy across runs :  80.63333333333334
Mean Acc for thrashed classes : 68.6


# Analysis of DHard and DHard_sub

In [13]:
# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_accs_dhard = []
all_stats_accs_dhard_sub = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_accs_dhard.append(stats['accuracy']['dhard_best'])
    all_stats_accs_dhard_sub.append(stats['accuracy']['dhard_sub_best'])

dhard_accs = list(all_stats_accs_dhard[0].values()) 
dhard_sub_accs = list(all_stats_accs_dhard_sub[0].values()) 
new_cols = [dhard_accs, dhard_sub_accs]
df_thrashed.insert(2, 'dhard_accs', dhard_accs)
df_thrashed.insert(3, 'dhard_sub_accs', dhard_sub_accs)


In [14]:
df_thrashed

,class,label,dhard_accs,dhard_sub_accs,accuracy_best_0
index,,,,,
ILSVRC2012_val_00000057.JPEG,Pomeranian,107,35.000000,0.0,74.0
ILSVRC2012_val_00000078.JPEG,"African hunting dog, hyena dog, Cape hunting d...",119,40.000000,25.0,94.0
ILSVRC2012_val_00000220.JPEG,"pug, pug-dog",102,11.764706,25.0,68.0
ILSVRC2012_val_00000350.JPEG,"Boston bull, Boston terrier",44,18.181818,50.0,50.0
ILSVRC2012_val_00000590.JPEG,"Samoyed, Samoyede",106,63.636364,75.0,90.0
ILSVRC2012_val_00000883.JPEG,Bedlington terrier,30,14.285714,0.0,60.0
ILSVRC2012_val_00001203.JPEG,Leonberg,103,62.500000,75.0,94.0
ILSVRC2012_val_00001626.JPEG,"borzoi, Russian wolfhound",18,32.142857,75.0,62.0
ILSVRC2012_val_00002298.JPEG,Welsh springer spaniel,67,28.125000,25.0,52.0


In [15]:
print("Mean dhard performance", df_thrashed['dhard_accs'].mean())
print("Mean dhard_sub performance", df_thrashed['dhard_sub_accs'].mean())
print("Mean acc_improvement", df_thrashed['accuracy_best_0'].mean())

Mean dhard performance 32.50809011459351
Mean dhard_sub performance 37.5
Mean acc_improvement 68.6
